In [1]:
BASE_FEATURES_FILE = "../all_stations/all_stations.csv"

## 2. Events
Extracted events from 2015-2019 from [London Events](http://www.londontown.com/London/London-Events-2019).</br>


**Assumption**: The district with the highest event density has the highest frequency.

In [ ]:
import json
import pandas as pd
with open('eventsdata/events-2015.json', 'r') as f:
    data15 = json.load(f)
df15= pd.DataFrame(data15)
df1 =df15.drop(['locality','url'],axis=1)
# Data from 2016
with open('eventsdata/events-2016.json', 'r') as f:
    data16 = json.load(f)
df16= pd.DataFrame(data16)
df2 = df16.drop(['locality','url'],axis=1)
# Data from 2017
with open('eventsdata/events-2017.json', 'r') as f:
    data17 = json.load(f)
df17= pd.DataFrame(data17)
df3 = df17.drop(['locality','url'],axis=1)
# Data from 2018
with open('eventsdata/events-2018.json', 'r') as f:
    data18 = json.load(f)
df18= pd.DataFrame(data18)
df4 = df18.drop(['locality','url'],axis=1)
# Data from 2019
with open('eventsdata/events-2019.json', 'r') as f:
    data19 = json.load(f)
df19= pd.DataFrame(data19)
df5 = df19.drop(['locality','url'],axis=1)

In [ ]:
# Merge data
frames = [df1, df2, df3,df4,df5]
final = pd.concat(frames)
#f = final.assign(Month='0')
final.title = final.title.map(lambda x: x.replace("\n", ""))
final.head()

In [ ]:
from NominatimLibrary import Locator
locator = Locator()
geonames = pd.read_csv("geoname_postcodes_world.txt", sep="\t", names=["country","postcode","city","","","","","","","lat","lon","accuracy"])

dico = {}
def locate(dictionnary, locator, x):
    if dico.get(x) == None:
        try:
            dico[x] = locator.get_coordinates("" + x + ", England, GB")
        except Exception as e:
            # print(e)
            # If it was not found, we use only the three first letters:
            dpt = x[:3]
            geo_coords = geonames[(geonames["country"] == "GB") & (geonames["postcode"] == dpt)][["lat","lon"]]
            # Try in geonames :
            if len(geo_coords) > 0:
                dico[x] = (geo_coords.iloc[0].lat, geo_coords.iloc[0].lon)
            else:
                # Finally, try with Nominatim with the three first characters:
                try:
                    dico[x] = locator.get_coordinates("" + dpt + ", England, GB")
                except Exception as e:
                    print("Could not find :", e)
                    if x[:3] == "XX1":
                        print("Note: it is normal the postcode doesn't exist, it is 'secret cinema' at an unknown location.")
                    # Finally if nothing is found, we take a point very far away:
                    dico[x] = (0, 0)
    return dico[x]

final["coords"] = final["postcode"].map(lambda x: locate(dico, locator, x))
final.head()

Fucking dataset, different formats for the dates:
- "day month year"
- "dow day month year"
- "day - day month year"
- "dow day - dow day month year"
- "day month - day month year"
- "day month to day month year"
- "Various Venues"
- "day and day month year"
- "day, day and day month year"
- "day, day, day and day month year"
- "day month, day month, day month and day month year"
- **Actually any combinations... !**

- Funny one : `15th May - 05th March 2016`

In [ ]:
import re
from datetime import date, timedelta

# It would have been possible to make it more generic, but...
SINGLE_DATE = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
TWO_DATES = "^[A-Za-z ]*([0-9]{1,2})[^ ]* and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
THREE_DATES = "^[A-Za-z ]*([0-9]{1,2})[^ ]*, [A-Za-z ]*([0-9]{1,2})[^ ]* and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
FOUR_DATES = "^[A-Za-z ]*([0-9]{1,2})[^ ]*, [A-Za-z ]*([0-9]{1,2})[^ ]*, [A-Za-z ]*([0-9]{1,2})[^ ]* and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"

TWO_MONTHS = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
THREE_MONTHS = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+), [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
FOUR_MONTHS = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+), [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+), [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) and [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"

SINGLE_DATE = "^[A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
DOUBLE_DAYS = "^([0-9]{1,2})[^ ]* (-|to) [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
DOUBLE_MONTHS = "^([0-9]{1,2})[^ ]* ([A-Za-z]+) (-|to) [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"
DOUBLE_YEARS = "^([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}) (-|to) [A-Za-z ]*([0-9]{1,2})[^ ]* ([A-Za-z]+) ([0-9]{4}).*"

MONTHS = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

def get_month_id(month):
    month_id = -1
    for (i,m) in enumerate(MONTHS):
        if m[:3] == month[:3]:
            month_id = i + 1
    return month_id

def make_date(match, year_id, month_id, day_id):
    year = int(match.group(year_id))
    month = get_month_id(match.group(month_id))
    day = int(match.group(day_id))
    return date(year,month,day)

def format_date(date_range):
    res = []
    start = None
    end = None

    if date_range == "Various Venues":
        return []
    
    match = re.match(DOUBLE_YEARS, date_range)
    if match != None:
        start = make_date(match,3,2,1)
        end = make_date(match,7,6,5)
    
    match = re.match(DOUBLE_MONTHS, date_range)
    if match != None:
        start = make_date(match,6,2,1)
        end = make_date(match,6,5,4)
    
    if start == None:   
        match = re.match(DOUBLE_DAYS, date_range)
        if match != None:
            start = make_date(match,5,4,1)
            end = make_date(match,5,4,3)
    
    if start != None:
        if start > end: # If only "day0 month0 - day1 month1 year" but month0 > month1, then 0 is a year before.
            start = start.replace(year=start.year - 1)
        while start <= end:
            res.append(start)
            start = start + timedelta(1)

    # If it is not a range:

    if len(res) == 0:
        match = re.match(SINGLE_DATE, date_range)
        if match != None:
            res.append(make_date(match,3,2,1))

    if len(res) == 0:
        match = re.match(TWO_DATES, date_range)
        if match != None:
            res.append(make_date(match,4,3,1))
            res.append(make_date(match,4,3,2))

    if len(res) == 0:
        match = re.match(THREE_DATES, date_range)
        if match != None:
            res.append(make_date(match,5,4,1))
            res.append(make_date(match,5,4,2))
            res.append(make_date(match,5,4,3))

    if len(res) == 0:
        match = re.match(FOUR_DATES, date_range)
        if match != None:
            res.append(make_date(match,6,5,1))
            res.append(make_date(match,6,5,2))
            res.append(make_date(match,6,5,3))
            res.append(make_date(match,6,5,4))
    
    if len(res) == 0:
        match = re.match(TWO_MONTHS, date_range)
        if match != None:
            res.append(make_date(match,5,2,1))
            res.append(make_date(match,5,4,3))

    if len(res) == 0:
        match = re.match(THREE_MONTHS, date_range)
        if match != None:
            res.append(make_date(match,7,2,1))
            res.append(make_date(match,7,4,3))
            res.append(make_date(match,7,6,5))

    if len(res) == 0:
        match = re.match(FOUR_MONTHS, date_range)
        if match != None:
            res.append(make_date(match,9,2,1))
            res.append(make_date(match,9,4,3))
            res.append(make_date(match,9,6,5))
            res.append(make_date(match,9,8,7))

    if len(res) == 0:
        print("Couldn't match: " + date_range)
    
    return res

final["dates_array"] = final["dates"].map(format_date)
final.head()
#format_date("15th May - 05th March 2016")

In [ ]:
events = final[["dates_array","title","coords"]]
events.to_csv("events_formatted.csv", sep=";")
events.head()

In [ ]:
#events = pd.read_csv('eventsPrepared.csv', sep=";")
#events.head(75)

In [ ]:
dates_events = {}

for index, row in events.iterrows():
    for d in row.dates_array:
        events_list = dates_events.get(d,[])
        events_list.append(row.coords)
        dates_events[d] = events_list

dates_events[date(2018,6,5)][:10]

In [ ]:
STATION_ID = 14
def get_station_coords(station_id):
    stations = pd.read_csv("../raw/rental_stations_saved.csv")

    coords = stations[(stations["id"] == STATION_ID)][["lat","lon"]].iloc[0]
    coords = (coords.lat, coords.lon)
    return coords

get_station_coords(STATION_ID)

In [ ]:
features = pd.read_csv(BASE_FEATURES_FILE)
features.head()

In [ ]:
NO_EVENT_DISTANCE = 9999

events_distances = []

for index, row in features.iterrows():
    row_date = date(row["Year"],row["Month"],row["Day of Month"])
    row_events = dates_events.get(row_date, [])
    closest_event_distance = NO_EVENT_DISTANCE
    if len(row_events) > 0:
        station_id = row["Station ID"]
        coords = get_station_coords(station_id)
        closest_event_distance = sorted(map(lambda x: locator.distance_crow_coords(coords, x), row_events))[0]
    
    events_distances.append(closest_event_distance)

features["nearestEvent"] = events_distances
features.head()

In [ ]:
features.to_csv("features_events.csv")